In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize

/home/naufalabdila/miniconda3/envs/jlml/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/naufalabdila/miniconda3/envs/jlml/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/naufalabdila/miniconda3/envs/jlml/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


# Read data 

In [2]:
df = pd.read_csv('./data/content_by_synopsis.csv')
df.head(2)

,title,overview
0,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,When siblings Judy and Peter discover an encha...


# Title to ID + ID to title conversion

Untuk kita ambil data menggunakan id atau title (mapping)

In [9]:
id2title = dict(df['title'])
title2id = {v:k for k, v in id2title.items()}

In [11]:
tfidf = TfidfVectorizer(stop_words='english', tokenizer=word_tokenize)
tfidf_matrix = tfidf.fit_transform(df['overview'])

# Cosine Similarity 

In [23]:
from sklearn.metrics.pairwise import cosine_distances

In [24]:
sim = cosine_distances(tfidf_matrix[title2id['Toy Story']], tfidf_matrix)
sim

array([[0.        , 0.96491002, 0.98518854, ..., 0.99647897, 0.97794662,
        0.99058318]])

In [25]:
# similarity terdekat dimulai dari angka yang dibelakang
sim.argsort()[0]

array([    0, 14706,  2945, ...,  9841, 38338, 15696])

In [26]:
rec = [id2title[i] for i in sim.argsort()[0, 1:11]]
rec

['Toy Story 3',
 'Toy Story 2',
 'The 40 Year Old Virgin',
 'Small Fry',
 "Andy Hardy's Blonde Trouble",
 'Andy Kaufman Plays Carnegie Hall',
 'Hot Splash',
 'Superstar: The Life and Times of Andy Warhol',
 'Andy Peters: Exclamation Mark Question Point',
 'Life Begins for Andy Hardy']

Kalau menggunakan synopsis atau overview dari suatu film sebagai parameter untuk merekomendasikan sesautu bisa jadi kurang valid.

# Sum up 

In [27]:
def content_based(df, title, matrix, id2title, title2id, topN):
    idx = title2id[title]
    sim = cosine_distances(matrix[idx], matrix)
    
    recommend_id = sim.argsort()[0, 1: topN+1]
    recommend_title = [id2title[i] for i in recommend_id]
    return recommend_title

In [29]:
title = 'Toy Story'

In [30]:
content_based(df=df, title=title, matrix=tfidf_matrix, id2title=id2title, title2id=title2id, topN=5)

['Toy Story 3',
 'Toy Story 2',
 'The 40 Year Old Virgin',
 'Small Fry',
 "Andy Hardy's Blonde Trouble"]

# Save model 

In [31]:
from jcopml.utils import save_model, load_model

In [33]:
model = {
    "matrix": tfidf_matrix,
    "id2title": id2title,
    "title2id": title2id
}

save_model(model, 'cont-based.pkl')

Model is pickled as model/cont-based.pkl


# Load model 

In [34]:
modelx = load_model('./model/cont-based.pkl')